# Web Server Logs Analysis

De forma general, un server log es un archivo de log generado por el servidor con una 
lista de las actividades que se ejecutan. En este caso tenemos un web server log el cuál 
mantiene un historial de las peticiones realizadas a la página. Este tipo de server logs 
tienen un formato standard (Common Log Format). Y es una práctica general, el 
analizar estos logs para sacar distintas conclusiones, localizar ataques, errores 
comunes, etc.


En nuestro caso tenemos el dataset de los web server logs de la NASA. Qué están compuestos por este tipo de registros:

*133.43.96.45 - - [01/Aug/1995:00:00:23 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1713*

Por lo que tenemos los siguientes campos:
1. Host: 133.43.96.45
2. User-identifier: en este dataset, todos estos campos estarán con un “-“ que significa que faltan esos datos, por lo que obviaremos este campo.
3. Userid: al igual que el anterior campo, también será obviado.
4. Date: 01/Aug/1995:00:00:23 -0400, como podemos ver está en formato dd/MMM/yyyy:HH:mm:ss y el campo final “-0400” sería el timezone que en este caso omitiremos, además haremos una transformación de los meses a forma numérica.
5. Request Method: GET, existen distintos métodos de petición.
6. Resource: /images/launch-logo.gif, sería el recurso al que se accede en esta petición.
7. Protocol: HTTP/1.0, y por ultimo en esta parte entre comillas tendríamos el protocolo utilizado al ser logs de 1995, seguramente sea el único protocolo utilizado.
8. HTTP status code: 200, existen distintos códigos de estado de HTTP en el link a continuación tienes más información: link
9. Size: 1713, y como ultimo campo tendríamos el tamaño del objeto recibido por el cliente en bytes. En casos de error del cliente, este campo no se encuentra por lo que al igual que en los userid, será indicado con un “-“, tenerlo en cuenta.


Ahora que ya entendemos que se encuentra dentro de nuestro web server log, vamos a pasar a analizarlo. Primero debemos cargar el archivo como un archivo de texto normal y realizar las transformaciones pertinentes, a la hora de limpiar y estructurar nuestro dataset utilizaremos expresiones regulares para recoger los campos que necesitamos. 


Guardaremos nuestro nuevo DataFrame ya estructurado en formato parquet. Y de este leeremos para realizar nuestro análisis



### Lectura de todos los logs

In [2]:
nasaText = spark.read.text("access_log_Jul95")

nasaText.show(5, False)

+-----------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                  |
+-----------------------------------------------------------------------------------------------------------------------+
|199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245                                 |
|unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985                      |
|199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085   |
|burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0               |
|199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179|
+-----------------------

### División de campos en diferentes columnas con expresiones regulares

def regexp_extract(e: Column, exp: String, groupIdx: Int): 

Column Extract a specific group matched by a Java regex, from the specified string column. If the regex did not match, or the specified group did not match, an empty string is returned.


### Expresiones regulares

https://regex101.com/r/78LS0X/1

In [3]:
from pyspark.sql.functions import regexp_extract

nasa_df = nasaText.select(regexp_extract('value', r'(\S+)', 1).alias('host'),
                         regexp_extract('value', r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2})', 1).alias('timestamp'),
                         regexp_extract('value', r'\"(\S+)\s(\S+)\s*(\S*)\"', 1).alias('method'),
                         regexp_extract('value', r'\"(\S+)\s(\S+)\s*(\S*)\"', 2).alias('endpoint'),
                         regexp_extract('value', r'\"(\S+)\s(\S+)\s*(\S*)\"', 3).alias('protocol'),
                         regexp_extract('value', r'\s(\d{3})\s', 1).cast('integer').alias('status'),
                         regexp_extract('value', r'\s(\S+)$', 1).cast('integer').alias('content_size'))
nasa_df.printSchema()

root
 |-- host: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- method: string (nullable = true)
 |-- endpoint: string (nullable = true)
 |-- protocol: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- content_size: integer (nullable = true)



In [12]:
nasa_df.show()

+--------------------+--------------------+------+--------------------+--------+------+------------+
|                host|           timestamp|method|            endpoint|protocol|status|content_size|
+--------------------+--------------------+------+--------------------+--------+------+------------+
|        199.72.81.55|01/Jul/1995:00:00:01|   GET|    /history/apollo/|HTTP/1.0|   200|        6245|
|unicomp6.unicomp.net|01/Jul/1995:00:00:06|   GET| /shuttle/countdown/|HTTP/1.0|   200|        3985|
|      199.120.110.21|01/Jul/1995:00:00:09|   GET|/shuttle/missions...|HTTP/1.0|   200|        4085|
|  burger.letters.com|01/Jul/1995:00:00:11|   GET|/shuttle/countdow...|HTTP/1.0|   304|           0|
|      199.120.110.21|01/Jul/1995:00:00:11|   GET|/shuttle/missions...|HTTP/1.0|   200|        4179|
|  burger.letters.com|01/Jul/1995:00:00:12|   GET|/images/NASA-logo...|HTTP/1.0|   304|           0|
|  burger.letters.com|01/Jul/1995:00:00:12|   GET|/shuttle/countdow...|HTTP/1.0|   200|    

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

nasa_clean_df = nasa_df.withColumn("host", when(col("host") == "", "Sin especificar").otherwise(col("host")))\
                        .withColumn("timestamp", to_timestamp(col("timestamp"), "dd/MMM/yyyy:HH:mm:ss"))\
                        .withColumn("method", when(col("method") == "", "Sin especificar").otherwise(col("method")))\
                        .withColumn("endpoint", when(col("endpoint") == "", "Sin especificar").otherwise(col("endpoint")))\
                        .withColumn("protocol", when(col("protocol") == "", "Sin especificar").otherwise(col("protocol")))

In [5]:
nasa_clean_df.printSchema()
nasa_clean_df.show(5, False)

root
 |-- host: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- endpoint: string (nullable = true)
 |-- protocol: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- content_size: integer (nullable = true)

+--------------------+-------------------+------+-----------------------------------------------+--------+------+------------+
|host                |timestamp          |method|endpoint                                       |protocol|status|content_size|
+--------------------+-------------------+------+-----------------------------------------------+--------+------+------------+
|199.72.81.55        |1995-07-01 00:00:01|GET   |/history/apollo/                               |HTTP/1.0|200   |6245        |
|unicomp6.unicomp.net|1995-07-01 00:00:06|GET   |/shuttle/countdown/                            |HTTP/1.0|200   |3985        |
|199.120.110.21      |1995-07-01 00:00:09|GET   |/shuttle/missions/sts-73/mis

In [6]:
nasa_clean_df.write.format("parquet")\
            .mode("overwrite")\
            .option("compression", "snappy")\
            .save("Parquet/nasa.parquet")

In [7]:
nasaParquet = spark.read.format("parquet").load("Parquet/nasa.parquet")

## Consultas

- ¿Cuáles son los distintos protocolos web utilizados? Agrúpalos.
- ¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos 
para ver cuál es el más común.
- ¿Y los métodos de petición (verbos) más utilizados?
- ¿Qué recurso tuvo la mayor transferencia de bytes de la página web?
- Además, queremos saber que recurso de nuestra web es el que más tráfico recibe. Es 
decir, el recurso con más registros en nuestro log.
- ¿Qué días la web recibió más tráfico?
- ¿Cuáles son los hosts son los más frecuentes?
- ¿A qué horas se produce el mayor número de tráfico en la web?
- ¿Cuál es el número de errores 404 que ha habido cada día?

In [22]:
# ¿Cuáles son los distintos protocolos web utilizados? Agrúpalos.
nasaParquet.select(col("protocol")).distinct().show()

+---------------+
|       protocol|
+---------------+
|Sin especificar|
|         HTTP/*|
|      HTTP/V1.0|
|       HTTP/1.0|
|  STS-69</a><p>|
+---------------+



In [23]:
# ¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos para ver cuál es el más común.

nasaParquet.select(col("status")).groupBy(col("status"))\
                                .agg(count(col("status")).alias("total"))\
                                .orderBy(desc(col("total")))\
                                .show()

+------+-------+
|status|  total|
+------+-------+
|   200|1701536|
|   304| 132627|
|   302|  46573|
|   404|  10843|
|   500|     62|
|   403|     54|
|   501|     14|
|   400|      5|
|  null|      0|
+------+-------+



In [41]:
# ¿Y los métodos de petición (verbos) más utilizados?

nasaParquet.select(col("method")).groupBy(col("method"))\
                                .agg(count(col("method")).alias("total"))\
                                .orderBy(desc(col("total")))\
                                .show()

+---------------+-------+
|         method|  total|
+---------------+-------+
|            GET|1886791|
|           HEAD|   3950|
|Sin especificar|    863|
|           POST|    111|
+---------------+-------+



In [46]:
# ¿Qué recurso tuvo la mayor transferencia de bytes de la página web?

nasaParquet.select(col("endpoint"),col("content_size"))\
                                .orderBy(desc(col("content_size")))\
                                .show(1, False)

+---------------------------------------+------------+
|endpoint                               |content_size|
+---------------------------------------+------------+
|/shuttle/countdown/video/livevideo.jpeg|6823936     |
+---------------------------------------+------------+
only showing top 1 row



In [49]:
# Además, queremos saber que recurso de nuestra web es el que más tráfico recibe. 
# Es decir, el recurso con más registros en nuestro log.

nasaParquet.select(col("endpoint")).groupBy(col("endpoint"))\
                                .agg(count(col("endpoint")).alias("total"))\
                                .orderBy(desc(col("total")))\
                                .show(1,False)

+--------------------------+------+
|endpoint                  |total |
+--------------------------+------+
|/images/NASA-logosmall.gif|111330|
+--------------------------+------+
only showing top 1 row



In [9]:
# ¿Qué días la web recibió más tráfico?

nasaParquet.select(col("timestamp")).groupBy(col("timestamp").cast("date").alias("Timestamp"))\
                                    .agg(count("*").alias("Trafico"))\
                                    .orderBy(desc(col("Trafico")))\
                                    .show(1)

+----------+-------+
| Timestamp|Trafico|
+----------+-------+
|1995-07-13| 134203|
+----------+-------+
only showing top 1 row



In [16]:
# ¿Cuáles son los hosts son los más frecuentes?

nasaParquet.select(col("host")).groupBy(col("host"))\
                                    .agg(count("*").alias("Visitas"))\
                                    .orderBy(desc(col("Visitas")))\
                                    .show(10)

+--------------------+-------+
|                host|Visitas|
+--------------------+-------+
|piweba3y.prodigy.com|  17572|
|piweba4y.prodigy.com|  11591|
|piweba1y.prodigy.com|   9868|
|  alyssa.prodigy.com|   7852|
| siltb10.orl.mmc.com|   7573|
|piweba2y.prodigy.com|   5922|
|  edams.ksc.nasa.gov|   5434|
|        163.206.89.4|   4906|
|         news.ti.com|   4863|
|disarray.demon.co.uk|   4353|
+--------------------+-------+
only showing top 10 rows



In [13]:
# ¿A qué horas se produce el mayor número de tráfico en la web?

nasaParquet.select(col("timestamp")).groupBy(hour(col("timestamp")).alias("Hours"))\
                                    .agg(count("*").alias("Trafico"))\
                                    .orderBy(desc(col("Hours")))\
                                    .show(24)

+-----+-------+
|Hours|Trafico|
+-----+-------+
|   23|  69362|
|   22|  70759|
|   21|  71922|
|   20|  69809|
|   19|  71776|
|   18|  79282|
|   17|  97609|
|   16| 118037|
|   15| 121200|
|   14| 122479|
|   13| 120814|
|   12| 122085|
|   11| 115720|
|   10| 105507|
|    9|  99969|
|    8|  83750|
|    7|  54017|
|    6|  35253|
|    5|  31919|
|    4|  32234|
|    3|  37398|
|    2|  45297|
|    1|  53066|
|    0|  62450|
+-----+-------+
only showing top 24 rows



In [23]:
# ¿Cuál es el número de errores 404 que ha habido cada día?

nasaParquet.select(col("timestamp"), col("status")).where(col("status") == 404)\
                                    .groupBy(hour(col("timestamp")).alias("Hours"))\
                                    .agg(count("*").alias("Errors"))\
                                    .orderBy(desc(col("Hours")))\
                                    .show(24)

+-----+------+
|Hours|Errors|
+-----+------+
|   23|   465|
|   22|   485|
|   21|   445|
|   20|   383|
|   19|   412|
|   18|   505|
|   17|   617|
|   16|   631|
|   15|   833|
|   14|   752|
|   13|   537|
|   12|   657|
|   11|   732|
|   10|   595|
|    9|   481|
|    8|   365|
|    7|   240|
|    6|   134|
|    5|   147|
|    4|   168|
|    3|   240|
|    2|   268|
|    1|   321|
|    0|   430|
+-----+------+

